In [10]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, GlobalAveragePooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from efficientnet import EfficientNetB3, center_crop_and_resize, preprocess_input

from keras import models
from keras import layers
from keras import callbacks

import time

import matplotlib.pyplot as plt

import cv2

In [11]:
BASEPATH = "stanford-dogs-dataset/"
BATCH_SIZE = 32
EPOCHS = 100

In [8]:
early_stopping = EarlyStopping(patience=10, verbose=1,restore_best_weights=True, monitor="val_acc")
reduce_lr = callbacks.ReduceLROnPlateau(factor=0.1, patience=3,verbose=1, monitor="val_acc")

In [13]:
def build_model(conv_base, trainable=False, dense_layers=2):
    model = models.Sequential()
    model.add(conv_base)
    model.add(layers.AveragePooling2D((5,5)))
    model.add(layers.Flatten())
    for _ in range(dense_layers):
        model.add(layers.Dense(2048))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))
    model.add(layers.Dense(512))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(120,activation='softmax'))
    
    if not trainable:
        for layer in conv_base.layers:
            layer.trainable = False

    model.compile("adam",loss="categorical_crossentropy",metrics=["accuracy"])

    print(model.summary())
    
    return model

In [15]:
conv_base=EfficientNetB3(weights='imagenet',include_top=False, input_shape=(224,224,3))

model = build_model(conv_base,dense_layers=5)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b3 (Model)      (None, 7, 7, 1536)        10783528  
_________________________________________________________________
average_pooling2d_3 (Average (None, 1, 1, 1536)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1536)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 2048)              3147776   
_________________________________________________________________
batch_normalization_241 (Bat (None, 2048)              8192      
_________________________________________________________________
dropout_7 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 2048)              4196352   
__________

# Using the ImageDataGenerator

In [23]:
train_datagen = ImageDataGenerator(rescale=1./255.0,
                                   shear_range=0.2,
                                   rotation_range=20.,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=[0.9, 1.25],
                                   brightness_range=[0.5, 1.5],
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255.0)

validation_datagen = ImageDataGenerator(rescale=1./255.0)

In [12]:
train_generator = train_datagen.flow_from_directory(
    BASEPATH + "train",
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    BASEPATH + "test",
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    BASEPATH + "val",
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

NameError: name 'train_datagen' is not defined

In [29]:
model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // BATCH_SIZE,
    validation_data = test_generator, 
    validation_steps = test_generator.samples // BATCH_SIZE,
    epochs = EPOCHS, callbacks=[early_stopping, reduce_lr])

Epoch 1/100
481/481 [==============================] - 419s 872ms/step - loss: 2.8884 - acc: 0.3184 - val_loss: 1.7625 - val_acc: 0.5211
Epoch 2/100
481/481 [==============================] - 354s 735ms/step - loss: 1.9695 - acc: 0.4841 - val_loss: 1.7767 - val_acc: 0.5416
Epoch 3/100
481/481 [==============================] - 320s 665ms/step - loss: 1.7027 - acc: 0.5326 - val_loss: 1.5228 - val_acc: 0.5850
Epoch 4/100
481/481 [==============================] - 411s 854ms/step - loss: 1.6040 - acc: 0.5600 - val_loss: 1.5074 - val_acc: 0.5924
Epoch 5/100
481/481 [==============================] - 331s 689ms/step - loss: 1.5330 - acc: 0.5768 - val_loss: 1.4964 - val_acc: 0.5915
Epoch 6/100
481/481 [==============================] - 323s 672ms/step - loss: 1.4380 - acc: 0.5981 - val_loss: 1.5954 - val_acc: 0.5823
Epoch 7/100
481/481 [==============================] - 330s 686ms/step - loss: 1.3753 - acc: 0.6229 - val_loss: 1.5610 - val_acc: 0.6294
Epoch 8/100
481/481 [====================

## Evaluation

In [30]:
loss, acc = model.evaluate_generator(validation_generator,verbose=0, steps=validation_generator.samples // batch_size)

In [31]:
print(loss,acc)

1.2640717449620014 0.734005905511811


In [32]:
model.save("efficientnetb3-model-{:.2f}acc-{:.2f}loss-{:.0f}.hdf5".format(acc, loss, time.time()))